In [ ]:
# !pip install git+https://github.com/keras-team/keras-applications.git -q

# import keras_applications as ka
# def set_to_tf(ka):
#     from tensorflow.keras import backend, layers, models, utils
#     ka._KERAS_BACKEND = backend
#     ka._KERAS_LAYERS = layers
#     ka._KERAS_MODELS = models
#     ka._KERAS_UTILS = utils
    
    
# set_to_tf(ka)

!pip install /kaggle/input/keras-pretrained-imagenet-weights/image_classifiers-1.0.0-py3-none-any.whl

from classification_models.tfkeras import Classifiers
Classifiers.models_names()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# ML tools 
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam
from tensorflow.keras import Model
# import tensorflow.keras.applications.efficientnet as efn
from tensorflow.keras.applications import *
import os
from keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [ ]:
df = pd.read_csv('../input/nih-dataframe/NIH_Dataframe.csv')
df.img_ind= df.img_ind.apply(lambda x: x.split('.')[0])
display(df.head(4))
print(df.shape)

In [ ]:
target_cols = df.drop(['img_ind'], axis=1).columns.to_list()
n_classes = len(target_cols)
img_size = 600
n_epochs = 35
lr= 0.0001
seed= 11
val_split= 0.2
seed= 33
batch_size= 32
n_classes

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

'''
Reference
https://www.kaggle.com/xhlulu/ranzcr-efficientnet-tpu-training

'''

def build_decoder(with_labels=True, target_size=(img_size, img_size), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path) # Reads and outputs the entire contents of the input filename.

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3) # Decode a PNG-encoded image to a uint8 or uint16 tensor
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3) # Decode a JPEG-encoded image to a uint8 tensor
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0 # Casts a tensor to the type float32 and divides by 255.
        img = tf.image.resize(img, target_size) # Resizing to target size
        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.1)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO) # overlaps data preprocessing and model execution while training
    return dset


In [ ]:
DATASET_NAME = "nih-image-600x600-data"
strategy = auto_select_accelerator()
batch_size = strategy.num_replicas_in_sync * batch_size
print('batch size', batch_size)

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path(DATASET_NAME)
GCS_DS_PATH

In [ ]:
paths = GCS_DS_PATH + "/NIH_Images/" + df['img_ind'] + '.jpg'


#Get the multi-labels
label_cols = df.columns[:-1]
labels = df[label_cols].values

In [ ]:
# Train test split
(train_paths, valid_paths, 
  train_labels, valid_labels) = train_test_split(paths, labels, test_size=val_split, random_state=11)

print(train_paths.shape, valid_paths.shape)
train_labels.sum(axis=0), valid_labels.sum(axis=0)

In [ ]:
valid

In [ ]:
valid.drop(['index'],inplace=True, axis = 1)

In [ ]:
valid

In [ ]:
valid['img_ind'][0]

In [ ]:
# Build the tensorflow datasets

decoder = build_decoder(with_labels=True, target_size=(img_size, img_size))

# Build the tensorflow datasets
dtrain = build_dataset(
    train_paths, train_labels, bsize=batch_size, decode_fn=decoder
)

dvalid = build_dataset(
    valid_paths, valid_labels, bsize=batch_size, 
    repeat=False, shuffle=False, augment=False, decode_fn=decoder
)

In [ ]:
data, _ = dtrain.take(2)
images = data[0].numpy()

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(20,10))
axes = axes.flatten()
for img, ax in zip(images, axes):
    ax.imshow(img)
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
def build_model():
    seresnet152, _ = Classifiers.get('seresnet152')
    base = seresnet152(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')
    
    inp = layers.Input(shape = (img_size, img_size, 3))
    x= base(inp)
    x= layers.GlobalAveragePooling2D()(layers.Dropout(0.16)(x))
    x= layers.Dropout(0.3)(x)
    x= layers.Dense(n_classes, 'sigmoid')(x)
    return Model(inp, x)
    

In [ ]:
with strategy.scope():
    model= build_model()
    loss= tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0)
    model.compile(optimizers.Adam(lr=lr),loss=loss,metrics=[tf.keras.metrics.AUC(multi_label=True)])

In [ ]:
model.summary()

In [ ]:
name= 'NIH_Seresnet152_model.h5'

rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 2, verbose = 1, 
                                min_delta = 1e-4, min_lr = 1e-6, mode = 'min', cooldown=1)
        
ckp = ModelCheckpoint(name,monitor = 'val_loss',
                      verbose = 1, save_best_only = True, mode = 'min')
        
es = EarlyStopping(monitor = 'val_loss', min_delta = 1e-4, patience = 5, mode = 'min', 
                    restore_best_weights = True, verbose = 1)

In [ ]:
steps_per_epoch = (train_paths.shape[0] // batch_size)
steps_per_epoch

In [ ]:
ensemble = model.fit(dtrain,                      
                    validation_data=dvalid,                                       
                    epochs=1,
                    callbacks=[rlr,es,ckp],
                    steps_per_epoch=steps_per_epoch,
                    verbose=1)

In [ ]:
from keras.models import load_model
import cv2
ensemble = load_model('../input/seresnet152-14-diseases-sigmoid/NIH_Seresnet152_model.h5')

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot( ensemble.history["loss"], label = "Training Loss", marker='o')
plt.plot( ensemble.history["val_loss"], label = "Validation Loss", marker='+')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot( ensemble.history["loss"], label = "Training Loss", marker='o')
plt.plot( ensemble.history["val_loss"], label = "Validation Loss", marker='+')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.plot( ensemble.history["auc"], label = "Training AUC" , marker='o')
plt.plot( ensemble.history["val_auc"], label = "Validation AUC", marker='+')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
tf.keras.backend.clear_session()

from sklearn.metrics import roc_auc_score

model= tf.keras.models.load_model(name)
pred= model.predict(dvalid, verbose=1)

print('AUC CKECK-UP per CLASS')

classes= df.columns[:-1]
for i, n in enumerate(classes):
  print(classes[i])
  print(i, roc_auc_score(valid_labels[:, i], pred[:, i]))
  print('---------')

In [ ]:
from keras.models import load_model
import cv2
denseNet_model = load_model('../input/densenet121-14-diseases-softmax/Densenet_model.h5')
# inceptionv2_model = load_model('../input/project-dataset/InceptionV2.h5')
mobilenet_model = load_model('../input/project-dataset/mobilenet.h5')

In [ ]:
disease_names = ["Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule","Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural Thickening","Hernia"]

In [ ]:
def DensepredictClass(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img/255
    img = cv2.resize(img, (256, 256))
    img = img.reshape((1,256,256,3))
    return denseNet_model.predict(img)

In [ ]:
def InceptionpredictClass(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img/255
    img = cv2.resize(img, (256, 256))
    img = img.reshape((1,256,256,3))
    return inceptionv2_model.predict(img)

In [ ]:
def mobilepredictClass(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img/255
    img = cv2.resize(img, (128, 128))
    img = img.reshape((1,128,128,3))
    return mobilenet_model.predict(img)

In [ ]:
disease_names = ["Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule","Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural Thickening","Hernia"]

In [ ]:
def classnames(arr):
    print(arr.size)
    for i in range(arr.size):
        print("Disease:", disease_names[i], " Probability:",arr[i]*100)

In [ ]:
def meanf(arr1, arr2, arr3):
    for i in range(14):
        print("Disease:", disease_names[i], " Probability:",((arr1[i]+0+arr3[i])/3) * 100)
      

In [ ]:
def soft_voting(path):
    x = DensepredictClass(path)
#     y = InceptionpredictClass(path)
    z = mobilepredictClass(path)
#     print("\nDenseNet")
#     classnames(x[0])
#     print("\nInceptionV2")
#     classnames(y[0])
#     print('\nMobileNet')
#     classnames(z[0])
    print('\nSoft Voting')
    return meanf(x[0], 0, z[0]) 

In [ ]:
import cv2

In [ ]:
#Prediction
path = "../input/data/images_001/images/00000001_000.png"
#Prediction
pred_result = soft_voting(path)

# For Accuracy

In [ ]:
test = pd.read_csv('../input/data/Data_Entry_2017.csv')
test.head()

In [ ]:
valid = valid_paths.reset_index()
valid.drop(['index'],inplace=True, axis = 1)
print(valid)

In [ ]:
valid['img_ind']

In [ ]:
df = pd.read_csv('../input/nih-dataframe/NIH_Dataframe.csv')
df.img_ind= df.img_ind.apply(lambda x: x.split('.')[0])
display(df.head(4))
print(df.shape)

In [ ]:
df = df.reset_index()
df.drop(['index'],inplace=True, axis = 1)
df.head(2)

In [ ]:
df.set_index('img_ind')
k = df[['img_ind', "Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule","Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural_Thickening","Hernia"]]


In [ ]:
result = []
for i in range(2):
    img = valid['img_ind'][i]
    img = '../input/nih-image-600x600-data/NIH_Images/' + img.split('/')[-1]
    result.append(soft_voting(img))
    

In [ ]:
img = valid['img_ind'][0]
# img = cv2.imread(img, cv2.IMREAD_COLOR)
# plt.imshow(img)
print(img)

In [ ]:
img = '../input/nih-image-600x600-data/NIH_Images/00014379_003.jpg'
img = cv2.imread(img, cv2.IMREAD_COLOR)
plt.imshow(img)

In [ ]:
l = 'gs://kds-6ba337acb78b30be6ad828a19720818407a57038a026c41e61df4852/NIH_Images/00014379_003.jpg'

In [ ]:
l.split('/')[-1]